In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
#from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
#from sklearn.metrics import ndcg_score
#from sklearn.metrics import label_ranking_average_precision_score
from matplotlib import pyplot

In [2]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))


def text_pre_processing(text):
     #remove punctuation
    text = remove_punctuation(text)
    
    #lowercase & remove extra whitespace
    text = " ".join(text.lower().split()) 
    
    return text

In [3]:
data = pd.read_csv("car_extra.csv", index_col=0) 
data['src_ad_id'] = data['src_ad_id'].apply(lambda x: '{:.0f}'.format(x))
#data['emissie'].fillna(0, inplace=True)
# test preprocessing for model field to combine the same filed
data['model_pro']= data['model'].apply(lambda text: text_pre_processing(text))
data.head(5).T

,0,1,2,3,4
src_ad_id,1123009959,1018323670,1092848988,1151159673,1061622715
telclicks,1,0,0,3,0
bids,0,0,0,0,0
kleur,Zilver of Grijs,Zilver of Grijs,Zilver of Grijs,Zwart,Zilver of Grijs
carrosserie,Hatchback (3/5-deurs),Hatchback (3/5-deurs),Stationwagon,Sedan (2/4-deurs),MPV
kmstand,44958,25072,301409,194481,238101
days_live,31,31,31,31,9
photo_cnt,16,22,12,21,8
aantaldeuren,3,5,5,4,4
n_asq,0,1,0,1,0


In [4]:
df = data[data['group']== 'B']
target_list = ['src_ad_id', 'telclicks', 'bids','n_asq','webclicks',
               'group','l2','ad_start_dt','date','nr_view','model'
               ,'bouwjaar','aantaldeuren','aantalstoelen','photo_cnt','kmstand']

df = df.loc[:, ~df.columns.isin(target_list)]
df.isnull().sum()

kleur                       0
carrosserie                 0
days_live                   0
emissie                     0
energielabel                0
brand                       0
vermogen                    0
price                       0
day_of_week                 0
auto_age                    0
has_view                    0
nr_door                     0
nr_chair                    0
nr_foto                     0
brand_nation                0
days_before_cyber_monday    0
has_kmstand                 0
l2_modified                 0
has_auto_driving            0
days_post                   0
vermogen_group              0
kmstand_pro                 0
new_auto                    0
price_zero                  0
model_pro                   0
dtype: int64

In [5]:
df.columns

Index(['kleur', 'carrosserie', 'days_live', 'emissie', 'energielabel', 'brand',
       'vermogen', 'price', 'day_of_week', 'auto_age', 'has_view', 'nr_door',
       'nr_chair', 'nr_foto', 'brand_nation', 'days_before_cyber_monday',
       'has_kmstand', 'l2_modified', 'has_auto_driving', 'days_post',
       'vermogen_group', 'kmstand_pro', 'new_auto', 'price_zero', 'model_pro'],
      dtype='object')

In [6]:
df[df['emissie']==0]['energielabel'].value_counts()

?    10619
C      805
A      477
B      473
D      390
E      161
G      146
F       89
Name: energielabel, dtype: int64

In [7]:
df['has_view'].describe()

count    80158.000000
mean         0.748497
std          0.433880
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: has_view, dtype: float64

## Target Encoding of Categorical Field
Categorical fields: <br>
    'kleur', 'carrosserie',  'energielabel', 'brand',
        'day_of_week', 'nr_door', 'nr_chair', 'nr_foto', 'brand_nation',  'days_post',
       'vermogen_group', 'model_pro'
<br>
<br>
Binary Fields (0, 1): <br>
'has_kmstand', 'has_auto_driving','new_auto', 'price_zero'
<br>
<br>
Numerical fields:<br>
 'days_live','emissie','vermogen', 'price','auto_age','days_before_cyber_monday','l2_modified','kmstand_pro'

In [8]:
msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]

In [9]:
import category_encoders as ce
cols = ['kleur','carrosserie','energielabel', 'brand', 'day_of_week'
        ,'nr_door', 'nr_chair', 'nr_foto', 'brand_nation','days_post', 'vermogen_group','model_pro']
#cols_enc =['kleur_enc','carrosserie_enc','energielabel_enc','brand_enc', 'day_of_week_enc'
#        ,'nr_door_enc', 'nr_chair_enc', 'nr_foto_enc', 'brand_nation_enc','days_post_enc'
#           , 'vermogen_group_enc','model_pro_enc']
target = 'has_view'
col_encoder = ce.LeaveOneOutEncoder(cols =cols)
col_encoder.fit(train_df[cols], train_df[target])
#train_df[cols_enc] = col_encoder.transform(train_df[cols])
train_df[cols] = col_encoder.transform(train_df[cols])
#test_df[cols_enc] = col_encoder.transform(test_df[cols])
test_df[cols] = col_encoder.transform(test_df[cols])

In [10]:
test_df.head().T

,6,17,23,40,55
kleur,0.742305,0.738962,0.776892,0.659119,0.770155
carrosserie,0.766307,0.700265,0.847590,0.763964,0.731146
days_live,2.000000,6.000000,31.000000,3.000000,12.000000
emissie,145.000000,195.000000,259.000000,0.000000,0.000000
energielabel,0.751210,0.830511,0.830511,0.830511,0.830511
brand,0.721268,0.690000,0.845233,0.812630,0.716615
vermogen,97.000000,80.000000,121.000000,65.000000,63.000000
price,269500.000000,89500.000000,25000.000000,25000.000000,10990.000000
day_of_week,0.774190,0.747006,0.760000,0.760000,0.718080
auto_age,1.000000,7.000000,16.000000,14.000000,18.000000


In [11]:
train_df.columns

Index(['kleur', 'carrosserie', 'days_live', 'emissie', 'energielabel', 'brand',
       'vermogen', 'price', 'day_of_week', 'auto_age', 'has_view', 'nr_door',
       'nr_chair', 'nr_foto', 'brand_nation', 'days_before_cyber_monday',
       'has_kmstand', 'l2_modified', 'has_auto_driving', 'days_post',
       'vermogen_group', 'kmstand_pro', 'new_auto', 'price_zero', 'model_pro'],
      dtype='object')

# Modelling

In [12]:
X_train = train_df.drop(['has_view'], axis=1).values
y_train = train_df['has_view'].values

X_test = test_df.drop(['has_view'], axis=1).values
y_test = test_df['has_view'].values

In [ ]:
from sklearn import ensemble
from sklearn import gaussian_process
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import svm
from sklearn import tree
from sklearn import discriminant_analysis
from xgboost import XGBClassifier
import lightgbm as lgb
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    
    #Naives Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    

    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier(),
    
    #LightGBM
    lgb.LGBMClassifier()
    ]

In [21]:
MLA_columns = ['MLA Name', 'MLA Parameters','Train ROC AUC','ROC AUC', 'F1-Score' ]
MLA_compare = pd.DataFrame(columns = MLA_columns)

#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    alg.fit(X_train, y_train)
    y_train_pred = alg.predict(X_train)
    y_pred = alg.predict(X_test)
    
    roc_auc_train = roc_auc_score(y_train, y_train_pred)   
    MLA_compare.loc[row_index, 'Train ROC AUC'] = roc_auc_train
    
    roc_auc = roc_auc_score(y_test, y_pred)   
    MLA_compare.loc[row_index, 'ROC AUC'] = roc_auc
    
    f1 = f1_score(y_test, y_pred, average='weighted')
    MLA_compare.loc[row_index, 'F1-Score'] = f1
    
    # calculate model precision-recall curve
    #precision, recall, _ = precision_recall_curve(y_test, y_pred)
    #auc_score = auc(recall, precision)  
    #MLA_compare.loc[row_index, 'PR AUC'] = auc_score
    
    row_index+=1

Overfitting :<br>
    -Random Forest Classifer (training ROC AUC >> test ROC AUC)

In [22]:
MLA_compare.sort_values(by = ['F1-Score'], ascending = False, inplace = True)
MLA_compare

,MLA Name,MLA Parameters,Train ROC AUC,ROC AUC,F1-Score
12,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.70682,0.676033,0.779721
4,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.99999,0.676157,0.776626
3,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.674423,0.661552,0.770954
2,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.99999,0.66688,0.769349
11,XGBClassifier,"{'base_score': 0.5, 'booster': 'gbtree', 'cols...",0.669507,0.65779,0.768899
0,AdaBoostClassifier,"{'algorithm': 'SAMME.R', 'base_estimator': Non...",0.667454,0.653322,0.763986
1,BaggingClassifier,"{'base_estimator': None, 'bootstrap': True, 'b...",0.990198,0.685447,0.759984
9,LinearDiscriminantAnalysis,"{'n_components': None, 'priors': None, 'shrink...",0.644114,0.632272,0.751539
7,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.99999,0.666292,0.741514
8,ExtraTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.99999,0.651342,0.734299


In [83]:
from flaml import AutoML
automl_settings = {
    "time_budget": 60*5,
    "metric": 'f1',
    "task": 'classification'
}
automl.fit(X_train = X_train, y_train = y_train,estimator_list=["lgbm"],
    **automl_settings)   ##automl.fit(X_train, y_train, task="classification", estimator_list=["lgbm"])

[flaml.automl: 06-06 00:48:53] {893} INFO - Evaluation method: holdout
[flaml.automl: 06-06 00:48:54] {596} INFO - Using StratifiedKFold
[flaml.automl: 06-06 00:48:54] {914} INFO - Minimizing error metric: 1-f1
[flaml.automl: 06-06 00:48:54] {934} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 06-06 00:48:54] {998} INFO - iteration 0, current learner lgbm
[flaml.automl: 06-06 00:48:54] {1150} INFO -  at 0.6s,	best lgbm's error=0.1434,	best lgbm's error=0.1434
[flaml.automl: 06-06 00:48:54] {998} INFO - iteration 1, current learner lgbm
[flaml.automl: 06-06 00:48:54] {1150} INFO -  at 0.8s,	best lgbm's error=0.1434,	best lgbm's error=0.1434
[flaml.automl: 06-06 00:48:54] {998} INFO - iteration 2, current learner lgbm
[flaml.automl: 06-06 00:48:54] {1150} INFO -  at 0.9s,	best lgbm's error=0.1332,	best lgbm's error=0.1332
[flaml.automl: 06-06 00:48:54] {998} INFO - iteration 3, current learner lgbm
[flaml.automl: 06-06 00:48:54] {1150} INFO -  at 1.0s,	best lgbm's erro

In [84]:
y_pred = automl.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred) 
f1 = f1_score(y_test, y_pred, average='weighted')
print('Best LightGBM ROC AUC %.3f' % roc_auc)
print('Best LightGBM F1 Score %.3f' % f1)

Best LightGBM ROC AUC 0.626
Best LightGBM F1 Score 0.754


In [85]:
automl.best_config

{'n_estimators': 17.0,
 'num_leaves': 224.0,
 'min_child_samples': 88.0,
 'learning_rate': 0.07564336600574059,
 'subsample': 0.97054170213439,
 'log_max_bin': 8.0,
 'colsample_bytree': 0.9141984327327011,
 'reg_alpha': 0.0018075418215889702,
 'reg_lambda': 0.06675273902338401,
 'FLAML_sample_size': 57638}

In [86]:
from collections import Counter
print(Counter(y_pred))

Counter({1: 14311, 0: 1804})


In [88]:
cl = lgb.LGBMClassifier()
cl.fit(X_train, y_train)
y_pred = cl.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print('Default LightGBM ROC AUC %.3f' % roc_auc)
print('Default LightGBM F1 Score %.3f' % f1)

Default LightGBM ROC AUC 0.676
Default LightGBM F1 Score 0.780


In [90]:
import pickle

# save the model to disk
filename = 'lgb_model.sav'
pickle.dump(cl, open(filename, 'wb'))


# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [91]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.7938566552901024


from collections import Counter
print(Counter(y_pred))